#!pip install dash_bootstrap_components
!pip install -U urllib3 requests


In [1]:
import os
import requests

In [2]:
cwd = os.getcwd()

In [3]:
import pandas as pd

In [4]:
import dash
from dash import dcc
from dash import html
from dash import dash_table

from dash.exceptions import PreventUpdate

from collections import OrderedDict

import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator
from plotly.subplots import make_subplots

import plotly.express as px

from dash import Dash, Input, Output,State, callback

import dash_bootstrap_components as dbc

In [5]:
from jupyter_dash import JupyterDash

In [6]:
PAGES_STYLE = "pagesstyles.css"
CONTENT_STYLES = "content.css"
external_stylesheets=[dbc.themes.BOOTSTRAP, PAGES_STYLE, CONTENT_STYLES]

app = JupyterDash(__name__,requests_pathname_prefix="/dash1/",routes_pathname_prefix='/dash1/',
                  external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [7]:
from dashboard_libraries import all_data_tables as adt
dashdataandtables = adt.DashDataAndTables()

In [8]:
#dashdataandtables.experiments_df

In [9]:
dashdataandtables.limits_data_df.head(5)

,id,limit_id,trace_id,trace_name,x,y,rowid
0,1374,1,1,d,30.0,0.0,0
1,1375,1,1,d,65.0,5.0,1
2,1376,1,1,d,90.0,25.0,2
3,1377,1,1,d,100.0,50.0,3
4,1378,1,1,d,90.0,75.0,4


# Color Code

In [10]:
from itertools import cycle

# colors
palette = cycle(px.colors.qualitative.Bold)
#palette = cycle(['black', 'grey', 'red', 'blue'])

# Scaling

In [11]:
from dashboard_libraries import scaling as sc
sf = sc.get_scale_factor('ub')
sf

9.999999999999999e-31

# Create Plot Series

In [12]:
def CreatePlotSeries(limits_in):
    plotseries_df = dashdataandtables.limits_traces_df[dashdataandtables.limits_traces_df['limit_id'].isin(limits_in)].copy()
    plotseries_df = plotseries_df.drop_duplicates()
    plotseries_df = plotseries_df.reset_index(drop=True)
    
    return plotseries_df


In [13]:
default_results = [1]
plotseries = CreatePlotSeries(default_results)
plotseries

,limit_id,trace_id,trace_name,rowid,line_color,line,fill_color,symbol,symbol_color
0,1,1,d,0,black,solid,LightGrey,square,blue
1,1,2,m,1,black,solid,LightGrey,square,blue
2,1,3,t,2,black,solid,LightGrey,square,blue
3,1,4,o1,3,black,solid,LightGrey,square,blue
4,1,5,o2,4,black,solid,LightGrey,square,blue
5,1,6,l,5,black,solid,LightGrey,square,blue


In [14]:
from dashboard_libraries import formattingtable as ft
default_limit = [1]
default_traces = dashdataandtables.limits_traces_df[dashdataandtables.limits_traces_df['limit_id']\
                                                    .isin(default_limit)].copy()
default_styledatatable = ft.CreateFormatTable(default_traces)

In [15]:
plotseries_default = CreatePlotSeries([1])
plotseries_default.head(5)

,limit_id,trace_id,trace_name,rowid,line_color,line,fill_color,symbol,symbol_color
0,1,1,d,0,black,solid,LightGrey,square,blue
1,1,2,m,1,black,solid,LightGrey,square,blue
2,1,3,t,2,black,solid,LightGrey,square,blue
3,1,4,o1,3,black,solid,LightGrey,square,blue
4,1,5,o2,4,black,solid,LightGrey,square,blue


In [16]:
from dashboard_libraries import createlegend as cl
default_limit = [1]
default_traces = dashdataandtables.limits_traces_df[dashdataandtables.limits_traces_df['limit_id']\
                                                    .isin(default_limit)].copy()
default_legend_fig = cl.CreateLegendFig(default_limit,dashdataandtables.limits_traces_df)

In [17]:
default_legend_out_graph = dcc.Graph(figure=default_legend_fig,id='legend_out_id',
                             style={'width': '100%', 'height': '100%'})

In [18]:
from dashboard_libraries import updatelegend as ul

In [19]:
from dashboard_libraries import getstyleandlegend as gsal

In [20]:
default_styleandlegendcolumn = gsal.GetStyleAndLegendColumn(default_styledatatable,default_legend_out_graph)

In [21]:
from dashboard_libraries import creategraph as cg
default_limit = [1]
default_traces = dashdataandtables.limits_traces_df[dashdataandtables.limits_traces_df['limit_id'].isin(default_limit)].copy()
default_graph_fig = cg.CreateGraph(default_limit,dashdataandtables.limits_traces_df,dashdataandtables.limits_data_df)

In [22]:
from dashboard_libraries import updategraph as ug

In [23]:
default_graph_out = dcc.Graph(figure=default_graph_fig,id='graph_out_id',
                            config=dict(responsive=True),
                            className='h-100'
                            #style={'width': '100%', 'height': '100%'}
                           )

In [24]:
chart_in = default_graph_out

def GetChart(chart_in):

    chartdiv = html.Div([chart_in], className="CHART_DIV NOPADDING")

    chart_col =  html.Div(children=[chartdiv],className="col col-lg-12 NOPADDING")

    chart_row = html.Div([chart_col],
                    className="row CHART_ROW NOPADDING")

    column_chart_out = html.Div(
                [chart_row]  ,
                className="col col-lg-6"
            )
    return column_chart_out

default_column_chart = GetChart(chart_in)

In [25]:
twocolumns =  html.Div(className="row g-0 ALL_ROW NOPADDING",children=[
                                                                    default_column_chart,
                                                                    default_styleandlegendcolumn,
                                                                      ],)

In [26]:
layout2 = html.Div([twocolumns],
                   className="container-fluid DASHBOARD_CONTAINER_STYLE",
                  )

In [27]:
app.layout = layout2

@app.callback(
    [Output("graph_out_id","figure"),Output("legend_out_id","figure"),],
    [Input('format_table_id', 'data')],
    [State("format_table_id", "data")])
def update_output(table_data, table_data_in):
    fig_out = ug.UpdateGraph(dashdataandtables, table_data_in)
    legend_out = ul.UpdateLegendFig(table_data_in)

    
    return fig_out, legend_out

In [28]:
app.run_server(host="0.0.0.0", port=8002, debug=True, use_reloader=False) 

Dash app running on http://0.0.0.0:8002/dash1/
